In [1]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height} at {Mode.bpp} bpp.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
face_recognition_cache = {}

def process_frame(frame):
    # Resize frame for faster face detection
    scale_factor = 0.5
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to RGB for face recognition
    frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    # Draw results on the original frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back the face locations to original frame size
        top *= (1 / scale_factor)
        right *= (1 / scale_factor)
        bottom *= (1 / scale_factor)
        left *= (1 / scale_factor)
        
        cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)
        cv2.rectangle(frame, (int(left), int(bottom) - 35), (int(right), int(bottom)), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (int(left), int(bottom) - 10), font, 1.0, (255, 255, 255), 1)

def process_and_display_video():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 2  # Process every 2nd frame

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            continue

        frame_count += 1
        if frame_count % frame_skip == 0:
            # Allocate a new frame for DisplayPort
            outframe = displayport.newframe()
            if outframe is None:
                print("Failed to allocate new frame.")
                continue

            # Process the frame (e.g., face detection and recognition)
            process_frame(frame)

            # Display the processed frame on the DisplayPort
            outframe[:] = frame
            try:
                displayport.writeframe(outframe)
            except Exception as e:
                print(f"Error writing processed frame: {e}")

        # Calculate and print FPS every second
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")
            frame_count = 0
            start_time = time.time()

try:
    # Start video processing
    process_and_display_video()

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()
    

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
pip install cmake


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 4.0 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dlib==19.17.0 --verbose


Using pip 22.0.2 from /usr/local/share/pynq-venv/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.6 MB/s eta 0:00:00:00:0100:01
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info
  writing /tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/dependency_links.txt
  writing top-level names to /tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/SOURCES.txt'
  package init file 'dlib/__init__.py' not found (or not a regular file)
  reading manifest file '/tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  writing manifest file '/tmp/pip-pip-egg-info-qszlt1ju/dlib.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Running command python setup.py bdist_whee

In [5]:
print("done")


done


In [ ]:
print("hello")


NameError: name 'python3' is not defined

In [6]:
print("hello")


hello


In [1]:
import face_recognition

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
pip install face-recognition


  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height} at {Mode.bpp} bpp.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
face_recognition_cache = {}

def process_frame(frame):
    # Resize frame for faster face detection
    scale_factor = 0.5
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to RGB for face recognition
    frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    # Draw results on the original frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back the face locations to original frame size
        top *= (1 / scale_factor)
        right *= (1 / scale_factor)
        bottom *= (1 / scale_factor)
        left *= (1 / scale_factor)
        
        cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)
        cv2.rectangle(frame, (int(left), int(bottom) - 35), (int(right), int(bottom)), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (int(left), int(bottom) - 10), font, 1.0, (255, 255, 255), 1)

def process_and_display_video():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 2  # Process every 2nd frame

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            continue

        frame_count += 1
        if frame_count % frame_skip == 0:
            # Allocate a new frame for DisplayPort
            outframe = displayport.newframe()
            if outframe is None:
                print("Failed to allocate new frame.")
                continue

            # Process the frame (e.g., face detection and recognition)
            process_frame(frame)

            # Display the processed frame on the DisplayPort
            outframe[:] = frame
            try:
                displayport.writeframe(outframe)
            except Exception as e:
                print(f"Error writing processed frame: {e}")

        # Calculate and print FPS every second
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")
            frame_count = 0
            start_time = time.time()

try:
    # Start video processing
    process_and_display_video()

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()

Error loading face encodings: list indices must be integers or slices, not str
Error configuring DisplayPort: 'VideoMode' object has no attribute 'bpp'
FPS: 0.52
FPS: 5.34


NameError: name 'known_face_encodings' is not defined

In [11]:
import os
import cv2
import pickle
import face_recognition

def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
    return images

def extract_face_encodings(images):
    encodings = []
    for image in images:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_image)
        face_encodings = face_recognition.face_encodings(rgb_image, face_locations)
        encodings.extend(face_encodings)
    return encodings

def save_encodings_to_file(encodings, names, output_file):
    data = {'encodings': encodings, 'names': names}
    with open(output_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"Saved {len(encodings)} face encodings to {output_file}")

if __name__ == "__main__":
    image_directory = 'ImageNet'  # Directory containing your images
    output_file = 'encoding_face.pkl'

    images = load_images_from_directory(image_directory)
    encodings = extract_face_encodings(images)
    names = ["Abheek"] * len(encodings)  # All images are labeled as "Abheek"
    save_encodings_to_file(encodings, names, output_file)

Saved 3 face encodings to encoding_face.pkl


In [12]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height} at {Mode.bpp} bpp.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
face_recognition_cache = {}

def process_frame(frame):
    # Resize frame for faster face detection
    scale_factor = 0.5
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to RGB for face recognition
    frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    # Draw results on the original frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back the face locations to original frame size
        top *= (1 / scale_factor)
        right *= (1 / scale_factor)
        bottom *= (1 / scale_factor)
        left *= (1 / scale_factor)
        
        cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)
        cv2.rectangle(frame, (int(left), int(bottom) - 35), (int(right), int(bottom)), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (int(left), int(bottom) - 10), font, 1.0, (255, 255, 255), 1)

def process_and_display_video():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 2  # Process every 2nd frame

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            continue

        frame_count += 1
        if frame_count % frame_skip == 0:
            # Allocate a new frame for DisplayPort
            outframe = displayport.newframe()
            if outframe is None:
                print("Failed to allocate new frame.")
                continue

            # Process the frame (e.g., face detection and recognition)
            process_frame(frame)

            # Display the processed frame on the DisplayPort
            outframe[:] = frame
            try:
                displayport.writeframe(outframe)
            except Exception as e:
                print(f"Error writing processed frame: {e}")

        # Calculate and print FPS every second
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")
            frame_count = 0
            start_time = time.time()

try:
    # Start video processing
    process_and_display_video()

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()

Error configuring DisplayPort: 'VideoMode' object has no attribute 'bpp'
FPS: 0.55
FPS: 5.36
FPS: 5.90
FPS: 2.17
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.96
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
Interrupted by user.


In [13]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

def process_frame(frame):
    # Resize frame for faster face detection
    scale_factor = 0.5
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to RGB for face recognition
    frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    # Draw results on the original frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back the face locations to original frame size
        top = int(top / scale_factor)
        right = int(right / scale_factor)
        bottom = int(bottom / scale_factor)
        left = int(left / scale_factor)
        
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

def process_and_display_video():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 2  # Process every 2nd frame

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            continue

        frame_count += 1
        if frame_count % frame_skip == 0:
            # Allocate a new frame for DisplayPort
            outframe = displayport.newframe()
            if outframe is None:
                print("Failed to allocate new frame.")
                continue

            # Process the frame (e.g., face detection and recognition)
            process_frame(frame)

            # Display the processed frame on the DisplayPort
            outframe[:] = frame
            try:
                displayport.writeframe(outframe)
            except Exception as e:
                print(f"Error writing processed frame: {e}")

        # Calculate and print FPS every second
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")
            frame_count = 0
            start_time = time.time()

try:
    # Start video processing
    process_and_display_video()

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


DisplayPort configured to 640x480.
FPS: 1.91
FPS: 6.99
FPS: 6.01
FPS: 6.99
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.94
FPS: 0.97
FPS: 0.94
FPS: 0.94
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 6.99
FPS: 6.01
FPS: 6.99
FPS: 6.01
FPS: 5.90
FPS: 6.99
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.94
FPS: 0.94
FPS: 0.97
FPS: 0.97
FPS: 0.94
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.94
FPS: 0.94
FPS: 0.94
FPS: 0.94
FPS: 0.94
FPS: 0.94
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
FPS: 0.97
Interrupted by user.


In [14]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

def process_frame(frame):
    # Resize frame for faster face detection
    scale_factor = 0.25
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to RGB for face recognition
    frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    # Draw results on the original frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back the face locations to original frame size
        top = int(top / scale_factor)
        right = int(right / scale_factor)
        bottom = int(bottom / scale_factor)
        left = int(left / scale_factor)
        
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

def process_and_display_video():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            continue

        frame_count += 1
        if frame_count % frame_skip == 0:
            # Allocate a new frame for DisplayPort
            outframe = displayport.newframe()
            if outframe is None:
                print("Failed to allocate new frame.")
                continue

            # Process the frame (e.g., face detection and recognition)
            process_frame(frame)

            # Display the processed frame on the DisplayPort
            outframe[:] = frame
            try:
                displayport.writeframe(outframe)
            except Exception as e:
                print(f"Error writing processed frame: {e}")

        # Calculate and print FPS every second
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")
            frame_count = 0
            start_time = time.time()

try:
    # Start video processing
    process_and_display_video()

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


DisplayPort configured to 640x480.
FPS: 1.87
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 18.45
FPS: 4.02
FPS: 2.68
FPS: 2.61
FPS: 2.61
FPS: 2.59
FPS: 18.38
FPS: 13.87
FPS: 14.07
FPS: 15.99
FPS: 14.09
FPS: 14.29
FPS: 15.99
FPS: 14.08
FPS: 2.52
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.59
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.61
FPS: 2.61
FPS: 2.59
FPS: 2.61
FPS: 2.59
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.70
FPS: 2.68
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
FPS: 2.68
FPS: 2.70
Interrupted by user.


In [ ]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time
from threading import Thread, Lock

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
frame_lock = Lock()
current_frame = None
face_recognition_results = []

def capture_frames():
    global current_frame
    while True:
        ret, frame = videoIn.read()
        if ret:
            with frame_lock:
                current_frame = frame.copy()

def process_frame():
    global current_frame, face_recognition_results
    scale_factor = 0.25
    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Resize frame for faster face detection
            small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

            # Convert frame to RGB for face recognition
            frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(frame_rgb)
            face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)

            face_recognition_results = (face_locations, face_names)

def display_frames():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Draw results on the original frame
            if face_recognition_results:
                face_locations, face_names = face_recognition_results
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back the face locations to original frame size
                    top = int(top / scale_factor)
                    right = int(right / scale_factor)
                    bottom = int(bottom / scale_factor)
                    left = int(left / scale_factor)
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Allocate a new frame for DisplayPort
                outframe = displayport.newframe()
                if outframe is None:
                    print("Failed to allocate new frame.")
                    continue

                # Display the processed frame on the DisplayPort
                outframe[:] = frame
                try:
                    displayport.writeframe(outframe)
                except Exception as e:
                    print(f"Error writing processed frame: {e}")

            # Calculate and print FPS every second
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.0:
                fps = frame_count / elapsed_time
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()

# Start threads for capturing, processing, and displaying frames
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frame, daemon=True)
display_thread = Thread(target=display_frames, daemon=True)

capture_thread.start()
process_thread.start()
display_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time
from threading import Thread, Lock

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
frame_lock = Lock()
current_frame = None
face_recognition_results = []

def capture_frames():
    global current_frame
    while True:
        ret, frame = videoIn.read()
        if ret:
            # Preprocess frame
            frame = preprocess_frame(frame)
            with frame_lock:
                current_frame = frame.copy()

def preprocess_frame(frame):
    # Resize frame for faster processing
    scale_factor = 0.25
    frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    # Convert frame to grayscale (optional, if face_recognition supports it)
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    return frame

def process_frame():
    global current_frame, face_recognition_results
    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Convert frame to RGB for face recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(frame_rgb)
            face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)

            face_recognition_results = (face_locations, face_names)

def display_frames():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Draw results on the original frame
            if face_recognition_results:
                face_locations, face_names = face_recognition_results
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back the face locations to original frame size
                    scale_factor = 4  # because we resized frame by 0.25
                    top = int(top * scale_factor)
                    right = int(right * scale_factor)
                    bottom = int(bottom * scale_factor)
                    left = int(left * scale_factor)
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Allocate a new frame for DisplayPort
                outframe = displayport.newframe()
                if outframe is None:
                    print("Failed to allocate new frame.")
                    continue

                # Display the processed frame on the DisplayPort
                outframe[:] = frame
                try:
                    displayport.writeframe(outframe)
                except Exception as e:
                    print(f"Error writing processed frame: {e}")

            # Calculate and print FPS every second
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.0:
                fps = frame_count / elapsed_time
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()

# Start threads for capturing, processing, and displaying frames
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frame, daemon=True)
display_thread = Thread(target=display_frames, daemon=True)

capture_thread.start()
process_thread.start()
display_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


DisplayPort configured to 640x480.
FPS: 0.51


Exception in thread Thread-7 (display_frames):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6645/4106332070.py", line 130, in display_frames
ValueError: could not broadcast input array from shape (120,160,3) into shape (480,640,3)


Interrupted by user.


In [ ]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time
from threading import Thread, Lock

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
frame_lock = Lock()
current_frame = None
face_recognition_results = []

def capture_frames():
    global current_frame
    while True:
        ret, frame = videoIn.read()
        if ret:
            # Preprocess frame
            frame = preprocess_frame(frame)
            with frame_lock:
                current_frame = frame.copy()

def preprocess_frame(frame):
    # Resize frame for faster processing
    scale_factor = 0.25
    frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    return frame

def process_frame():
    global current_frame, face_recognition_results
    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Convert frame to RGB for face recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(frame_rgb)
            face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)

            face_recognition_results = (face_locations, face_names)

def display_frames():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Draw results on the original frame
            if face_recognition_results:
                face_locations, face_names = face_recognition_results
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back the face locations to original frame size
                    scale_factor = 4  # because we resized frame by 0.25
                    top = int(top * scale_factor)
                    right = int(right * scale_factor)
                    bottom = int(bottom * scale_factor)
                    left = int(left * scale_factor)
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            # Resize frame back to original size
            frame = cv2.resize(frame, (frame_in_w, frame_in_h))

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Allocate a new frame for DisplayPort
                outframe = displayport.newframe()
                if outframe is None:
                    print("Failed to allocate new frame.")
                    continue

                # Display the processed frame on the DisplayPort
                outframe[:] = frame
                try:
                    displayport.writeframe(outframe)
                except Exception as e:
                    print(f"Error writing processed frame: {e}")

            # Calculate and print FPS every second
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.0:
                fps = frame_count / elapsed_time
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()

# Start threads for capturing, processing, and displaying frames
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frame, daemon=True)
display_thread = Thread(target=display_frames, daemon=True)

capture_thread.start()
process_thread.start()
display_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time
from threading import Thread, Lock

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
frame_lock = Lock()
current_frame = None
face_recognition_results = []

def capture_frames():
    global current_frame
    while True:
        ret, frame = videoIn.read()
        if ret:
            # Preprocess frame
            frame = preprocess_frame(frame)
            with frame_lock:
                current_frame = frame.copy()

def preprocess_frame(frame):
    # Resize frame for faster processing
    scale_factor = 0.25
    frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)

    return frame

def process_frame():
    global current_frame, face_recognition_results
    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Convert frame to RGB for face recognition
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(frame_rgb)
            face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)

            face_recognition_results = (face_locations, face_names)

def display_frames():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Draw results on the original frame
            if face_recognition_results:
                face_locations, face_names = face_recognition_results
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back the face locations to original frame size
                    scale_factor = 4  # because we resized frame by 0.25
                    top = int(top * scale_factor)
                    right = int(right * scale_factor)
                    bottom = int(bottom * scale_factor)
                    left = int(left * scale_factor)
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            # Resize frame back to original size
            frame = cv2.resize(frame, (frame_in_w, frame_in_h))

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Allocate a new frame for DisplayPort
                outframe = displayport.newframe()
                if outframe is None:
                    print("Failed to allocate new frame.")
                    continue

                # Display the processed frame on the DisplayPort
                outframe[:] = frame
                try:
                    displayport.writeframe(outframe)
                except Exception as e:
                    print(f"Error writing processed frame: {e}")

            # Calculate and print FPS every second
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.0:
                fps = frame_count / elapsed_time
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()

# Start threads for capturing, processing, and displaying frames
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frame, daemon=True)
display_thread = Thread(target=display_frames, daemon=True)

capture_thread.start()
process_thread.start()
display_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


DisplayPort configured to 640x480.
FPS: 0.47
FPS: 0.60
FPS: 1.72
FPS: 0.56
FPS: 0.58
FPS: 1.15
FPS: 0.56
FPS: 0.58
FPS: 1.15
FPS: 1.10
FPS: 0.57
FPS: 0.58
FPS: 0.60
FPS: 1.14
FPS: 1.11
FPS: 0.54
FPS: 1.15
FPS: 1.15
FPS: 1.10
FPS: 0.58
FPS: 0.58
FPS: 0.58
FPS: 0.60
FPS: 1.15
FPS: 0.58
FPS: 0.56
FPS: 0.29
FPS: 1.10
FPS: 2.29
FPS: 1.15
FPS: 7.97
FPS: 9.93
FPS: 8.65
FPS: 7.94
FPS: 8.99
FPS: 8.31
FPS: 8.10
FPS: 7.61
FPS: 2.24
FPS: 0.61
FPS: 0.58
FPS: 1.10
FPS: 1.09
FPS: 2.85
FPS: 0.58
FPS: 1.09
FPS: 0.56
FPS: 0.56
FPS: 0.58
FPS: 1.65
FPS: 0.57
FPS: 1.10
FPS: 1.10
FPS: 0.57
FPS: 1.14
FPS: 0.61
FPS: 0.56
FPS: 0.56
FPS: 1.11
FPS: 1.59
FPS: 0.57
FPS: 0.57
FPS: 0.58
FPS: 0.58
FPS: 1.06
FPS: 0.57
FPS: 0.55
FPS: 0.60
Interrupted by user.
FPS: 1.09


In [ ]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time
from threading import Thread, Lock

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
    exit(1)
except Exception as e:
    print(f"Error loading face encodings: {e}")
    exit(1)

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
    exit(1)

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")
    exit(1)

# Shared data
frame_lock = Lock()
current_frame = None
face_recognition_results = []

def capture_frames():
    global current_frame
    while True:
        ret, frame = videoIn.read()
        if ret:
            with frame_lock:
                current_frame = frame.copy()

def process_frame():
    global current_frame, face_recognition_results
    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Resize frame for faster processing
            scale_factor = 0.25
            small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
            
            # Convert frame to RGB for face recognition
            frame_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(frame_rgb)
            face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                face_names.append(name)

            face_recognition_results = (face_locations, face_names, scale_factor)

def display_frames():
    frame_count = 0
    start_time = time.time()  # Start time for FPS calculation
    frame_skip = 5  # Process every 5th frame

    while True:
        if current_frame is not None:
            with frame_lock:
                frame = current_frame.copy()
            
            # Draw results on the original frame
            if face_recognition_results:
                face_locations, face_names, scale_factor = face_recognition_results
                for (top, right, bottom, left), name in zip(face_locations, face_names):
                    # Scale back the face locations to original frame size
                    top = int(top / scale_factor)
                    right = int(right / scale_factor)
                    bottom = int(bottom / scale_factor)
                    left = int(left / scale_factor)
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    font = cv2.FONT_HERSHEY_DUPLEX
                    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Allocate a new frame for DisplayPort
                outframe = displayport.newframe()
                if outframe is None:
                    print("Failed to allocate new frame.")
                    continue

                # Display the processed frame on the DisplayPort
                outframe[:] = frame
                try:
                    displayport.writeframe(outframe)
                except Exception as e:
                    print(f"Error writing processed frame: {e}")

            # Calculate and print FPS every second
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.0:
                fps = frame_count / elapsed_time
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()

# Start threads for capturing, processing, and displaying frames
capture_thread = Thread(target=capture_frames, daemon=True)
process_thread = Thread(target=process_frame, daemon=True)
display_thread = Thread(target=display_frames, daemon=True)

capture_thread.start()
process_thread.start()
display_thread.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()


DisplayPort configured to 640x480.
FPS: 0.50
FPS: 1.14
FPS: 1.09
FPS: 1.12
FPS: 1.64
FPS: 3.76
FPS: 0.52
FPS: 1.17
FPS: 1.09
FPS: 1.12
FPS: 1.09
FPS: 1.12
FPS: 2.18
FPS: 1.12
FPS: 2.18
FPS: 0.56
FPS: 1.38
FPS: 1.58
FPS: 0.54
FPS: 1.58
FPS: 1.08
FPS: 1.64
FPS: 1.67
FPS: 0.57
FPS: 0.56
FPS: 1.64
FPS: 1.63
FPS: 8.12
FPS: 0.58
FPS: 0.58
FPS: 1.59


In [ ]:
import cv2
import numpy as np
import face_recognition
from pynq.lib.video import DisplayPort, VideoMode, PIXEL_RGB
import pickle
import time

# Load face encodings and names
try:
    with open('encoding_face.pkl', 'rb') as f:
        data = pickle.load(f)
        known_face_encodings = data['encodings']
        known_face_names = data['names']
except FileNotFoundError:
    print("File not found: encoding_face.pkl")
except Exception as e:
    print(f"Error loading face encodings: {e}")

# Camera configuration
frame_in_w = 640
frame_in_h = 480
fps = 30

videoIn = cv2.VideoCapture(0 + cv2.CAP_V4L2)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
videoIn.set(cv2.CAP_PROP_FPS, fps)

if not videoIn.isOpened():
    print("Failed to open camera.")
else:
    print("Camera opened successfully.")

# DisplayPort configuration
Mode = VideoMode(frame_in_w, frame_in_h, 24)
displayport = DisplayPort()

try:
    displayport.configure(Mode, PIXEL_RGB)
    print(f"DisplayPort configured to {Mode.width}x{Mode.height}.")
except Exception as e:
    print(f"Error configuring DisplayPort: {e}")


In [ ]:
def preprocess_frame(frame):
    scale_factor = 0.5
    small_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
    return small_frame

def process_frame(frame, known_face_encodings, known_face_names):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

    return face_locations, face_names

def draw_results(frame, face_locations, face_names, scale_factor=0.5):
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top = int(top / scale_factor)
        right = int(right / scale_factor)
        bottom = int(bottom / scale_factor)
        left = int(left / scale_factor)

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    return frame


In [ ]:
# Capture, process, and display video frames
try:
    frame_count = 0
    start_time = time.time()

    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to read from camera.")
            break

        small_frame = preprocess_frame(frame)
        face_locations, face_names = process_frame(small_frame, known_face_encodings, known_face_names)
        draw_results(frame, face_locations, face_names)

        outframe = displayport.newframe()
        if outframe is not None:
            np.copyto(outframe, frame)
            displayport.writeframe(outframe)
        else:
            print("Failed to allocate new frame.")

        frame_count += 1
        if frame_count % fps == 0:
            elapsed_time = time.time() - start_time
            fps_value = frame_count / elapsed_time
            print(f"FPS: {fps_value:.2f}")
            frame_count = 0
            start_time = time.time()

except KeyboardInterrupt:
    print("Interrupted by user.")
finally:
    videoIn.release()
    displayport.close()
    cv2.destroyAllWindows()
